In [1]:
from cgmbrush import *
from tempfile import TemporaryFile
import cProfile
import io
import pstats
from scipy.ndimage.filters import convolve
import scipy.signal as sig

In [ ]:
# Perf testing for reading Bolshoi density fields

provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
den_field = provider.get_density_field(0, 256)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

saveArray("perf_denfield", den_field)

In [ ]:
provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
read_denfield = loadArray("perf_denfield.npy")
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

file_path = os.path.join(varFolder, 'bolshoi_density_perf_read.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

In [ ]:
# Testing savez
outfile = TemporaryFile()
x = np.arange(1000)
y = np.arange(1000) + 1

arrays = {'y':y, 'x':x, }

np.savez(outfile, **arrays)
_ = outfile.seek(0)
npzfile = np.load(outfile)
npzfile['x']

In [ ]:
p = BolshoiProvider()
a = p.extract_halos(0)
print(a.shape)
b = loadArray('bol_halos_0.npy')
print(b.shape)

In [ ]:
print(a.shape)
print(a.info(memory_usage='deep'))
print(b.shape)
print(type(b))
print(str((b.size * b.itemsize)/1024**2) + " MB")
print(b[0])

In [2]:
# Profiling full convolution code for simplest case

min_mass = 10**10 # TODO into config
max_mass = 10**14.5
log_bins = 30
config = Configuration('tophat_spherical', 1, resolution=1, file_prefix='STH', load_from_files=False)
config.provider = BolshoiProvider()
filename = config.file_prefix + str(config.resolution) + '_' + str(config.den_grid_size)


#pr = cProfile.Profile()
#pr.enable()
hist_profile(config.provider, config.den_grid_size, config.RS_array, min_mass, 
                                    max_mass, log_bins, config.subtraction_halo_profile, 
                                    config.addition_halo_profile, config.scaling_radius, config.resolution)
#pr.disable()
#s = io.StringIO()
#ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
#ps.print_stats()

#file_path = os.path.join(varFolder, 'perf_convo_' + filename + '_v3.txt')
#with open(file_path, 'w') as f:
#    f.write(s.getvalue())

#print(config.results)
#saveArray(filename, config.results)
     




/Users/ianw89/Repos/cgm-brush/src/cgmbrush.py:455: RuntimeWarning: invalid value encountered in double_scalars
  return integrate.quad(lambda x, y, z: rho_nought/(((offset+(x**2+y**2+ z**2)**.5)/R_s)*(1+((x**2+y**2+ z**2)**.5)/R_s)**2),-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0]
/Users/ianw89/Repos/cgm-brush/src/cgmbrush.py:455: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  return integrate.quad(lambda x, y, z: rho_nought/(((offset+(x**2+y**2+ z**2)**.5)/R_s)*(1+((x**2+y**2+ z**2)**.5)/R_s)**2),-1*np.real((Rvir**2-(x**2+y**2))**.5),np.real((Rvir**2-(x**2+y**2))**.5),args=(x,y))[0]
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/numpy/lib/function_base.py:2246: RuntimeWarning: invalid value encountered in NFW2D (vectorized)
  outputs = ufunc(*inputs)


4
Starting async tasks
All async tasks complete
(29, 2048, 2048)
0.00012753715242798874
(29, 2048, 2048)
0.00012753715242798874
Addition profiles after add_halos returned: 
[[ 1.92705313e-14  1.55396524e+00  2.12409452e-14 ... -1.94561670e-14
   1.55396524e+00  1.17159084e+01]
 [ 1.55396524e+00  6.84856768e+00  6.95587608e+00 ...  3.28640283e-14
   6.95587608e+00  4.25644766e-14]
 [ 3.72244235e+00  8.83184108e+00  2.49996628e+00 ...  7.14531838e+00
   1.03590958e+01  2.88028162e-15]
 ...
 [ 6.84856768e+00  4.86734070e+00  3.40321974e+00 ...  3.72244235e+00
   2.40582776e+01  1.55396524e+00]
 [-8.15973968e-15 -3.63876522e-14  6.42130594e+00 ...  8.83184108e+00
   2.01344081e+01 -3.67949535e-14]
 [ 6.95587608e+00  1.55396524e+00  1.17159084e+01 ...  5.66315093e-15
  -6.11257916e-15  1.55396524e+00]]
halosremoved_fine: 
[[nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 ...
 [nan nan nan ... nan nan nan]
 [nan nan nan ... nan nan nan]
 [nan nan n

ValueError: supplied range of [0, nan] is not finite

In [ ]:
# Profiling full convolution code for simplest case
min_mass = 10**10 # TODO into config
max_mass = 10**14.5
log_bins = 30
config = Configuration('tophat_spherical', 1, resolution=16, file_prefix='STH', load_from_files=False)
config.provider = BolshoiProvider()
filename = config.file_prefix + str(config.resolution) + '_' + str(config.den_grid_size)


pr = cProfile.Profile()
pr.enable()
config.results = hist_profile(config.provider, config.den_grid_size, config.RS_array, min_mass, 
                                    max_mass, log_bins, config.subtraction_halo_profile, 
                                    config.addition_halo_profile, config.scaling_radius, config.resolution)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

saveArray(filename, config.results)

file_path = os.path.join(varFolder, 'perf_convo_' + filename + '_v2.txt')
with open(file_path, 'w') as f:
    f.write(s.getvalue())

SCRATCH WORK BELOW HERE

In [ ]:
a = np.random.random((1024,1024))
b = np.random.random((20,20))

In [ ]:
# This is the convolve function we are using today, but in a mode where the big array is given 0's outside the edge of the grid
# This is 'wrong' for our application but it makes for an easy comparison with the next function, which doesn't support 
# the reflect mode we use.
c1 = convolve(a, b, mode='constant', origin=-1)

In [ ]:
# The scipy signal implementation is much faster. But it only supports 0's around the edge
c2 = sig.convolve(a, b, mode='same')
np.allclose(c1,c2)

In [ ]:
# This is almost what we do today: just same method but using wrap instead of reflect
c7 = convolve(a, b, mode='wrap', origin=-1) # a b c d | a b c d | a b c d
c8 = convolve(a, b, mode='wrap')

In [ ]:
c3 = my_convolve(a, b)
print(np.allclose(c7,c3))
print(np.allclose(c8,c3)) # My implementation is equivalent to what we did before!


In [ ]:
# This is what we are actually running today. The same code as c1 but in reflect mode. d c b a | a b c d | d c b a
c4 = convolve(a, b, origin=-1)
print(np.allclose(c4,c3))
print(np.allclose(c4,c1))     

In [ ]:
# Testing signal's convolve 2d. It's too slow.
c5 = sig.convolve2d(a, b, mode='same', boundary='symm')
print(np.allclose(c5,c1))
print(np.allclose(c5,c2))
print(np.allclose(c5,c3))
print(np.allclose(c5,c4)) # interestingly it does match what we dotoday exactly

In [ ]:
# Valid mode basically trims the edges. This is proof that using signal in this is equivalent to what we are doing today,
# but throwing away a tiny bit of data!
c6 = sig.convolve(a, b, mode='valid')
print(np.allclose(c1[40:-39, 40:-39], c6))
print(np.allclose(c4[40:-39, 40:-39], c6))
print(np.allclose(c4[39:-40, 39:-40], c6))